In [39]:
!pip install autocorrect


## Step 1: Preparing the environment

In [40]:
 

import pandas as pd
import re
import pickle


from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from nltk.corpus import stopwords
from autocorrect import Speller
import sklearn
import numpy as np
import matplotlib as plt
#import pycaret
import nltk
import spacy

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.20.1.


In [41]:
!pip install simpletransformers
!pip install spacy
!pip install scikit-learn==0.20.1

## Step 2: Text Pre-processing

In [42]:

def spelling(text):
    
    spell = Speller(lang='en')
    return str(spell(text))
    
def tokening(text):
    
    #nltk.download('punkt')
    tokenized_word=word_tokenize(text)
    return tokenized_word
    
def removing_stopwords(text):
    
    #nltk.download('punkt')
    tokenized_word1=tokening(text)
    #nltk.download('stopwords')
    stop_words=set(stopwords.words("english"))
    filtered_sent=[]
    for w in tokenized_word1:
        if w not in stop_words:
            filtered_sent.append(w)
    return " ".join(filtered_sent)



def Lemmating(text):
    #nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    tokenized_word1=tokening(text)
    stemmed_words=[]
    for w in tokenized_word1:
        stemmed_words.append(lemmatizer.lemmatize(w))
    return " ".join(stemmed_words)

def pos_tag(text):
    #nltk.download('averaged_perceptron_tagger')
    tokenized_word1=tokening(text)
    pos=nltk.pos_tag(removing_stopwords(text1))
    return pos
    
# function called for cleaning html tags from text
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [44]:
# import data
data = pd.read_csv('datapoint_33.csv')
data.head()

,CompanyName,Point,QuoteText,QuoteDoc,LaymanTerm,Index Topic
0,scratch,neutral,<p>\n<i>The Scratch Privacy Policy was last up...,Privacy Policy,There is a date of the last update of the terms,Notice of Changing Terms
1,trello,bad,\n</ul>Sharing with third parties <p>We share ...,Privacy Policy,Third parties may be involved in operating the...,Third Parties
2,trello,bad,"Send ""Do Not Track"" Signals:</b> Some browsers...",Privacy Policy,This service ignores the Do Not Track (DNT) he...,Cookies
3,trello,good,You have the right to request a copy of your i...,Trello Privacy Policy,You can request access and deletion of persona...,NaN
4,trello,neutral,These Cloud Terms of Service will go into effe...,Atlassian Cloud Terms of Service,There is a date of the last update of the terms,Notice of Changing Terms


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140 entries, 0 to 2139
Data columns (total 6 columns):
CompanyName    2140 non-null object
Point          2140 non-null object
QuoteText      2140 non-null object
QuoteDoc       2140 non-null object
LaymanTerm     2140 non-null object
Index Topic    909 non-null object
dtypes: object(6)
memory usage: 100.4+ KB


In [46]:
data=data.dropna()
data.reset_index(drop=True)

,CompanyName,Point,QuoteText,QuoteDoc,LaymanTerm,Index Topic
0,scratch,neutral,<p>\n<i>The Scratch Privacy Policy was last up...,Privacy Policy,There is a date of the last update of the terms,Notice of Changing Terms
1,trello,bad,\n</ul>Sharing with third parties <p>We share ...,Privacy Policy,Third parties may be involved in operating the...,Third Parties
2,trello,bad,"Send ""Do Not Track"" Signals:</b> Some browsers...",Privacy Policy,This service ignores the Do Not Track (DNT) he...,Cookies
3,trello,neutral,These Cloud Terms of Service will go into effe...,Atlassian Cloud Terms of Service,There is a date of the last update of the terms,Notice of Changing Terms
4,trello,bad,5.4.</strong>\n<u>Your Indemnity</u>. You will...,Atlassian Cloud Terms of Service,"You agree to defend, indemnify, and hold the s...",Governance
...,...,...,...,...,...,...
904,imdb,bad,"As we continue to develop our business, we mig...",Privacy Policy,The service can sell or otherwise transfer you...,Business Transfers
905,imdb,bad,We use web beacons (also known as “action tags...,Privacy Policy,"The service may use tracking pixels, web beaco...",Cookies
906,imdb,good,Promotional Offers</b>: Sometimes we send offe...,Privacy Policy,You can opt out of promotional communications,Governance
907,imdb,blocker,IMDb has the right but not the obligation to m...,Terms of Service,The service can delete specific content withou...,Ownership


In [47]:
data.isnull().sum()

CompanyName    0
Point          0
QuoteText      0
QuoteDoc       0
LaymanTerm     0
Index Topic    0
dtype: int64

In [48]:
nltk.download('punkt')
sentences = []
for row in data.itertuples():
    for sentence in sent_tokenize(row[3]):
        if type(sentence) != float:
            if sentence != '':
                sentences.append((row[6], sentence))
data_sentenced = pd.DataFrame(sentences, columns=['Index Topic', 'Sentence'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nayefaboutayoun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
row

Pandas(Index=2138, CompanyName='imdb', Point='neutral', QuoteText='Last Updated, February 8, 2018</b>', QuoteDoc='Privacy Policy', LaymanTerm='There is a date of the last update of the terms', _6='Notice of Changing Terms')

In [51]:
data_sentenced['Sentence'] = data_sentenced.apply(lambda x: re.sub('[^0-9a-zA-Z]+', ' ', x['Sentence']), axis=1)
data_sentenced['Sentence'] = data_sentenced.apply(lambda x: re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", x['Sentence']), axis=1)
#remove all tags like <p> etc. in the data frame "data_sentenced"
data_sentenced['Sentence'] = data_sentenced['Sentence'].str.lower()
data_sentenced['Sentence'] = data_sentenced.apply(lambda x: cleanhtml(x['Sentence']), axis=1)
#remove stop words
nltk.download('stopwords')
data_sentenced['Sentence'] = data_sentenced.apply(lambda x: removing_stopwords(x['Sentence']), axis=1)
#Lemmaing words
nltk.download('wordnet')

data_sentenced['Sentence'] = data_sentenced.apply(lambda x: Lemmating(x['Sentence']), axis=1)
data_sentenced['Index Topic'].unique()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nayefaboutayoun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nayefaboutayoun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


array(['Notice of Changing Terms', 'Third Parties', 'Cookies',
       'Governance', 'User information', 'Guarantee',
       'Right to leave the service', 'Jurisdiction and governing laws',
       'Personal Data', 'Ownership', 'Changes',
       'Suspension and Censorship', 'Scope of the copyright license',
       'Business Transfers', 'Anonymity and Tracking', 'User choice',
       'Waiving your right'], dtype=object)

In [64]:
data_sentenced.describe()

,Index Topic
count,1581.000000
mean,6.148640
std,4.246225
min,0.000000
25%,2.000000
50%,6.000000
75%,10.000000
max,16.000000


In [65]:
data_sentenced.head()

,Index Topic,Sentence
0,0,p scratch privacy policy last updated may 2018
1,1,ul sharing third party p share information thi...
2,1,service provider need access information perfo...
3,1,may also share information third party agreed ...
4,1,may give third party apps access account infor...


In [66]:
l=[]
i=0
for topic in data_sentenced['Index Topic'].unique ():
  data_sentenced['Index Topic'].replace(topic, i, inplace=True)
  i=i+1
  l.append(topic,)
data_sentenced

,Index Topic,Sentence
0,0,p scratch privacy policy last updated may 2018
1,1,ul sharing third party p share information thi...
2,1,service provider need access information perfo...
3,1,may also share information third party agreed ...
4,1,may give third party apps access account infor...
...,...,...
1576,3,promotional offer b sometimes send offer selec...
1577,3,give business name e mail address
1578,3,want receive offer please use user administrat...
1579,9,imdb right obligation monitor edit remove acti...


In [67]:
len(data_sentenced.loc[data_sentenced['Index Topic']==5])

36

In [68]:
data_sentenced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1581 entries, 0 to 1580
Data columns (total 2 columns):
Index Topic    1581 non-null int64
Sentence       1581 non-null object
dtypes: int64(1), object(1)
memory usage: 24.8+ KB


## Step 3: Creating Training and Testing Dataset 

In [69]:
X, y = data_sentenced['Sentence'], data_sentenced['Index Topic']

X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
train_dataset =  pd.concat([X_train, y_train], axis=1)
train_dataset

,Sentence,Index Topic
1227,case error report data generate,8
930,brainly make warranty brainly service product ...,5
1520,agree expense indemnify defend hold harmless n...,3
236,ryver licensors supplier disclaim warranty whe...,5
1343,however please note system ever completely secure,5
...,...,...
1130,may modify contract privacy policy cooky polic...,10
1294,collect strong p p information necessary use s...,14
860,third party may collect information online act...,2
1459,p creating account platform free opt receiving...,3


## Step 4: Building the Model using roBerta 

In [75]:
from simpletransformers.classification import ClassificationModel


# Create a Roberta + distillation  
args={'output_dir': 'outputs/','reprocess_input_data': True, 'max_seq_length': 300, 'num_train_epochs': 4, 'fp16': False, 'train_batch_size': 2, 'overwrite_output_dir': True}

roberta_model = ClassificationModel('roberta', 'distilroberta-base', num_labels=17, use_cuda=False, cuda_device=0, args=args)

# Train the model
roberta_model.train_model(train_dataset)

# Evaluate the model
#result, model_outputs, wrong_predictions = roberta_model.eval_model(eval_dataset, acc=sklearn.metrics.f1_score)

Running loss: 1.036182


Running loss: 0.005815


Running loss: 0.001643


Running loss: 0.785353



## Step 5: Evaluating the Model

In [76]:
eval_dataset = pd.concat([X_val, y_val], axis=1)
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = roberta_model.eval_model(eval_dataset, f1=f1_multiclass, acc=accuracy_score)

In [77]:
result

{'mcc': 0.781887092763435,
 'f1': 0.804416403785489,
 'acc': 0.804416403785489,
 'eval_loss': 1.0577090288948967}

## Step 6: Saving the Trained Model Parameters

In [79]:
#Saving the classification 
with open('roberta_model.pickle','wb') as f:
    pickle.dump(roberta_model,f)
